In [1]:
#finds site map of entire domain
from usp.tree import sitemap_tree_for_homepage

#for saving and loading objects
import pickle

#for accessing url html
import requests
import urllib.request, urllib.parse, urllib.error
from urllib.request import Request, urlopen

#for html parsing
from bs4 import BeautifulSoup

#anxillary functions/methods
import numpy as np
import pandas as pd
import time
import random
import datetime
from dateutil.tz import tzutc, tzoffset

pd.set_option('display.max_colwidth', -1)
from tqdm import tqdm, trange
import time

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# Sitemap

In [ ]:
#SOUTH CHINA MORNING POST
# scmp articles as of date 4/11/2020. 
tree = sitemap_tree_for_homepage('https://www.scmp.com/hk');

In [ ]:
urls = []
dates = []
for page in tree.all_pages():
    urls.append(page.url)
    dates.append(page.last_modified)

In [ ]:
#transpose
zippedList =list(zip(urls,dates))

#format into dataframe
df = pd.DataFrame(zippedList, columns = ['urls' ,'date'])

#drop nan for date published and drop duplicates
df = df.dropna()
df = df.drop_duplicates()

#df.to_pickle("../Pickles/SCMP_All_URLs.pkl")


In [ ]:
with open('../../Ignore/SCMP_All_URLs.pkl', 'rb') as f:
    df = pickle.load(f)
    
    

# Data Scrape

In [2]:
#LOAD DATASET OF ALL SCRAPED URLS FROM SCMP
with open('../../Ignore/SCMP_All_URLs.pkl', 'rb') as f:
    df = pickle.load(f)

#drop 14 articles before 2012. 2012 onwards there are many more articles (5k articles in 2012 alone)
df = df.loc[(df['date']> pd.Timestamp('2012-1-1 00:00:00+0000', tz= tzutc()))]

#find articles with /news/ to find news articles only
df = df[df.urls.str.contains("/news/")]

#take out 4512 Young Post articles
df = df[~df.urls.str.contains("/yp/")]

#take out Photos articles - there's one
df = df[~df.urls.str.contains("/photos/")]

#take out Gallery articles - there's one
df = df[~df.urls.str.contains("/gallery/")]

#sort to date
df = df.sort_values(by=['date'])

#reset df index
df = df.reset_index(drop = True)

df

,urls,date
0,https://www.scmp.com/news/18th-party-congress/power-game/article/1001459/security-tsar-will-attend-party-congress-after,2012-05-19 16:00:00+00:00
1,https://www.scmp.com/news/18th-party-congress/challenges/article/1006828/legal-policy-chief-urges-better-petition,2012-07-14 16:00:00+00:00
2,https://www.scmp.com/news/hong-kong/article/1007255/peng-chau-industrial-hive-turned-rural-idyll,2012-07-19 16:00:00+00:00
3,https://www.scmp.com/news/hong-kong/article/1008735/play-our-strength,2012-08-12 13:59:00+00:00
4,https://www.scmp.com/news/hong-kong/article/1008741/hk-leader-leung-must-focus-building-team-controversies-mount,2012-08-12 14:17:00+00:00
...,...,...
172390,https://www.scmp.com/news/hong-kong/health-environment/article/3109853/coronavirus-hong-kongs-executive-council-signs,2020-11-14 15:30:00+00:00
172391,https://www.scmp.com/news/china/diplomacy/article/3109825/pope-francis-said-be-final-stage-selecting-new-hong-kong,2020-11-14 15:30:00+00:00
172392,https://www.scmp.com/news/world/united-states-canada/article/3109887/after-assailing-fox-news-calling-arizona-bidens,2020-11-14 15:39:00+00:00
172393,https://www.scmp.com/news/hong-kong/law-and-crime/article/3109867/hong-kongs-controversial-national-security-law-tip,2020-11-14 15:47:00+00:00


In [10]:
author = []
title = []
summary = []
link = []
topic = []
date = []

broken = []

In [11]:
#using BS4 to request each html data
for i in tqdm(range(160000,172395)):
    try:
        time.sleep(1) 
        req = Request(str(df.urls[i]), headers={'User-Agent': 'Potato'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')

        by = soup.find("meta", {"name": "cse_author"})
        ti = soup.find("meta", {"name": "cse_headline"})
        to = soup.find("meta", {"name": "cse_topic"})
        summ = soup.find("meta", {"name": "cse_summary"})


        if by and ti:
            author.append(by["content"])
            title.append(ti["content"])
            topic.append(to["content"])
            summary.append(summ["content"])
            link.append(df.urls[i])
            date.append(df.date[i])
            
            
            if len(link)% 1000 == 0:
                print("Dataset contains " + str(len(link)) + " entries.")
                print("Number of broken URLs: " + str(len(broken)))
    except:
        print("URL number " + str(i) + " is broken: " + df.urls[i] )
        broken.append(df.urls[i])
        pass


  8%|▊         | 1002/12395 [36:09<4:55:04,  1.55s/it]

Dataset contains 1000 entries.
Number of broken URLs: 0


 10%|▉         | 1180/12395 [41:58<13:59:28,  4.49s/it]

URL number 161179 is broken: https://www.scmp.com/news/asia/southeast-asia/article/3077473/medical-evacuation-plane-crashes-manila-airport


 16%|█▌        | 2004/12395 [1:09:14<4:33:00,  1.58s/it] 

...

 89%|████████▉ | 11030/12395 [6:14:20<51:33,  2.27s/it]  

Dataset contains 11000 entries.
Number of broken URLs: 3


 97%|█████████▋| 12034/12395 [6:41:30<09:06,  1.51s/it]  

Dataset contains 12000 entries.
Number of broken URLs: 3


100%|██████████| 12395/12395 [6:50:17<00:00,  1.99s/it]


In [12]:
scmp_df = pd.DataFrame(list(zip(title, author, summary, topic,link,date)), 
               columns =['Title', 'Author', 'Summary','Topic','Link', 'Date']) 

scmp_df.to_pickle("../Pickles/SCMP_News172395.pkl")

In [2]:
!ls ../Pickles/
#scmp_df


SCMP_News10000.pkl      SCMP_News1700000.pkl    SCMP_News80000.pkl
SCMP_News100000.pkl     SCMP_News20000.pkl      SCMP_News90000.pkl
SCMP_News110000.pkl     SCMP_News26000.pkl      SCMP_News_Sample_DF.pkl
SCMP_News120000.pkl     SCMP_News30000.pkl      SCMP_Opinion_URLS.p
SCMP_News130000.pkl     SCMP_News40000.pkl      SCMP_Opinions.pkl
SCMP_News140000.pkl     SCMP_News50000.pkl      ScmpDF.pkl
SCMP_News150000.pkl     SCMP_News60000.pkl
SCMP_News160000.pkl     SCMP_News70000.pkl


# Combine Datasets and Clean

In [120]:
with open('../Pickles/SCMP_News10000.pkl', 'rb') as f:
    df = pickle.load(f)
    
for i in range(20000,170001,10000):
    string = f'../Pickles/SCMP_News{i}.pkl'
    with open(string, 'rb') as f:
        df_add = pickle.load(f)
        
        df = df.append(df_add, ignore_index = True)
        
#remove empty author entries
df = df[df['Author'] != '']


In [121]:
clean_list = ["Associated Press", 
"AP", 
"AFP",
"Agence France-Presse", 
"Reuters",  
"The Guardian", 
"Guardian News Service",
"Agencies", 
"Bloomberg", 
"GUARDIAN",
"The Washington Post",
"WASHPOST",
"@scmp.com",
"McClatchy",
"MCT",
"MCY",
"NYT",
"Straits Times",
"POLITICO",
"#Tech in Asia",
"Xinhua",
"WIRE",
"Science magazine",
"USA TODAY",
"Tribune News Service",
".com",]

In [122]:
for i in clean_list:
    df = df[~df.Author.str.contains(i)]

In [123]:
df = df.reset_index(drop = True)
df

,Title,Author,Summary,Topic,Link,Date
0,Play to our strength,Chan Kin Wa,,,https://www.scmp.com/news/hong-kong/article/1008735/play-our-strength,2012-08-12 13:59:00+00:00
1,HK leader Leung must focus on building team as controversies mount,Tammy Tam,,Leung Chun Ying,https://www.scmp.com/news/hong-kong/article/1008741/hk-leader-leung-must-focus-building-team-controversies-mount,2012-08-12 14:17:00+00:00
2,Loh a curious choice for government role but could make an impact in office,Tony Cheung,,"Donald Tsang,Donald Tsang",https://www.scmp.com/news/hong-kong/article/1008737/loh-curious-choice-government-role-could-make-impact-office,2012-08-12 15:57:00+00:00
3,HK policeman Andy Naylor dies in water at Ironman triathlon in New York,Patsy Moy,,,https://www.scmp.com/news/hong-kong/article/1008742/hk-policeman-andy-naylor-dies-water-ironman-triathlon-new-york,2012-08-12 16:59:00+00:00
4,Why so glum? Hongkongers need to savour city's success,"MICHALE AU,LILLY ZHANG",,,https://www.scmp.com/news/hong-kong/article/1008740/why-so-glum-hongkongers-need-savour-citys-success,2012-08-12 23:13:00+00:00
...,...,...,...,...,...,...
92246,"Hong Kong police inspector arrested on suspicion of stealing colleagues’ tactical helmets, abalone",Christy Leung,The disappearance of the tactical gear and canned seafood from the offices of the public relations bureau prompted the man’s colleagues to file a complaint.,"Hong Kong police,Crime in Hong Kong",https://www.scmp.com/news/hong-kong/law-and-crime/article/3109872/hong-kong-police-inspector-arrested-suspicion-stealing,2020-11-14 15:28:00+00:00
92247,"Mandatory Covid-19 testing for ‘high-risk’ groups in Hong Kong gets nod, those who ignore orders face jail, fines","Elizabeth Cheung,Gigi Choy,Chris Lau",Other new measures include reintroduction of four-person cap on tables at eateries.,"Coronavirus pandemic,Coronavirus pandemic: All stories,Coronavirus Hong Kong,Hong Kong social distancing,Executive Council of Hong Kong",https://www.scmp.com/news/hong-kong/health-environment/article/3109853/coronavirus-hong-kongs-executive-council-signs,2020-11-14 15:30:00+00:00
92248,Pope Francis said to be in final stage of selecting new Hong Kong bishop,Mimi Lau,"Pontiff has three nominations to consider after an earlier announcement was postponed, according to informed source.","China-Vatican relations,Catholic Church,Hong Kong national security law (NSL),Pope Francis",https://www.scmp.com/news/china/diplomacy/article/3109825/pope-francis-said-be-final-stage-selecting-new-hong-kong,2020-11-14 15:30:00+00:00
92249,"Hong Kong’s controversial national security law tip line gets 10,000 messages in first week of operation",Christy Leung,"By 8am on Thursday, the multi-platform project had gathered the equivalent of 1,400 piece of information a day for the force’s new national security unit.","Hong Kong national security law (NSL),Hong Kong police,Hong Kong protests",https://www.scmp.com/news/hong-kong/law-and-crime/article/3109867/hong-kongs-controversial-national-security-law-tip,2020-11-14 15:47:00+00:00


# Test Set

In [124]:
df.to_pickle("../Pickles/SCMP_News_Full.pkl")    